In [6]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from helpers import *

# from test_utils import test

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:

x ,y ,ids = load_csv("dataset/x_train.csv", "dataset/y_train.csv")

# Valeurs
print(x)


[[5.3000000e+01 1.1000000e+01 1.1162015e+07 ...           nan
            nan 2.0000000e+00]
 [3.3000000e+01 1.2000000e+01 1.2152015e+07 ...           nan
            nan           nan]
 [2.0000000e+01 1.0000000e+01 1.0202015e+07 ... 1.0000000e+00
  2.0000000e+00 2.0000000e+00]
 ...
 [3.9000000e+01 1.0000000e+01 1.0202015e+07 ... 2.0000000e+00
  2.0000000e+00 2.0000000e+00]
 [3.3000000e+01 1.2000000e+01 1.2302015e+07 ...           nan
            nan 2.0000000e+00]
 [3.2000000e+01 9.0000000e+00 9.1220150e+06 ...           nan
            nan 2.0000000e+00]]


In [8]:
# 0 ou 1 
print(y)

[0 0 0 ... 0 0 0]


In [9]:
# List des participants 

print(ids)

range(0, 328135)
